<a href="https://colab.research.google.com/github/ShrutiJain456/compute/blob/main/bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

importing data

In [107]:
data = '/content/churndata_processed.csv'
df = pd.read_csv(data)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   months                     7043 non-null   float64
 1   multiple                   7043 non-null   int64  
 2   gb_mon                     7043 non-null   float64
 3   security                   7043 non-null   int64  
 4   backup                     7043 non-null   int64  
 5   protection                 7043 non-null   int64  
 6   support                    7043 non-null   int64  
 7   unlimited                  7043 non-null   int64  
 8   contract                   7043 non-null   float64
 9   paperless                  7043 non-null   int64  
 10  monthly                    7043 non-null   float64
 11  satisfaction               7043 non-null   float64
 12  churn_value                7043 non-null   int64  
 13  payment_Credit Card        7043 non-null   int64

In [109]:
df.columns

Index(['months', 'multiple', 'gb_mon', 'security', 'backup', 'protection',
       'support', 'unlimited', 'contract', 'paperless', 'monthly',
       'satisfaction', 'churn_value', 'payment_Credit Card',
       'payment_Mailed Check', 'internet_type_DSL',
       'internet_type_Fiber Optic', 'internet_type_None', 'offer_Offer A',
       'offer_Offer B', 'offer_Offer C', 'offer_Offer D', 'offer_Offer E'],
      dtype='object')

In [110]:
df.duplicated().sum()

92

In [111]:
df.drop_duplicates(inplace = True)

In [112]:
df.duplicated().sum()

0

In [113]:
df.isnull().sum()

months                       0
multiple                     0
gb_mon                       0
security                     0
backup                       0
protection                   0
support                      0
unlimited                    0
contract                     0
paperless                    0
monthly                      0
satisfaction                 0
churn_value                  0
payment_Credit Card          0
payment_Mailed Check         0
internet_type_DSL            0
internet_type_Fiber Optic    0
internet_type_None           0
offer_Offer A                0
offer_Offer B                0
offer_Offer C                0
offer_Offer D                0
offer_Offer E                0
dtype: int64

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6951 entries, 0 to 7042
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   months                     6951 non-null   float64
 1   multiple                   6951 non-null   int64  
 2   gb_mon                     6951 non-null   float64
 3   security                   6951 non-null   int64  
 4   backup                     6951 non-null   int64  
 5   protection                 6951 non-null   int64  
 6   support                    6951 non-null   int64  
 7   unlimited                  6951 non-null   int64  
 8   contract                   6951 non-null   float64
 9   paperless                  6951 non-null   int64  
 10  monthly                    6951 non-null   float64
 11  satisfaction               6951 non-null   float64
 12  churn_value                6951 non-null   int64  
 13  payment_Credit Card        6951 non-null   int64

In [115]:
df['churn_value'].value_counts()

0    5084
1    1867
Name: churn_value, dtype: int64

In [116]:
from sklearn.utils import resample
df_majority=df[df['churn_value']==0]
df_minority=df[df['churn_value']==1]
df_new=resample(df_minority,replace=True,n_samples=5084,random_state=50)

In [117]:
df1=pd.concat([df_new,df_majority])

In [118]:
df1['churn_value'].value_counts()

1    5084
0    5084
Name: churn_value, dtype: int64

In [119]:
y=df1.loc[:,'churn_value'].values
X=df1.drop(['churn_value'],axis=1).values

splitting and training the data

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.22, random_state = 20)

In [130]:
model = RandomForestClassifier(n_estimators = 10, random_state = 0, criterion = 'gini')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

Since the data is imbalanced, we can use f1 score as it combines precision and recall.

In [131]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report,accuracy_score

In [132]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1084,   35],
       [  25, 1093]])

In [133]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1119
           1       0.97      0.98      0.97      1118

    accuracy                           0.97      2237
   macro avg       0.97      0.97      0.97      2237
weighted avg       0.97      0.97      0.97      2237



In [134]:
bg = BaggingClassifier(base_estimator = RandomForestClassifier(), random_state = 20)
bg.fit(X_train, y_train)
y_bg_pred = bg.predict(X_test)

In [135]:
from sklearn import model_selection
result = model_selection.cross_val_score(model, X, y)
print(result.mean())

0.9762982083720868


In [136]:
y_pred_train = bg.predict(X_train)
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3965
           1       0.99      1.00      0.99      3966

    accuracy                           0.99      7931
   macro avg       0.99      0.99      0.99      7931
weighted avg       0.99      0.99      0.99      7931



the model fits perfectly